In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
plt.style.use('seaborn')

from sklearn.model_selection import KFold, train_test_split
from torchsummary import summary

## utils

In [2]:
class dfDataset(Dataset):
    def __init__(self, x, y):
        self.data = x
        self.target = y
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.target[index]

In [3]:
def E1_loss(y_pred, y_true):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: distance error normalized with 2e+04
    '''
    
    _t, _p = y_true, y_pred
    
    return torch.mean(torch.mean((_t - _p) ** 2, axis = 1)) / 2e+04

def E2_loss(y_pred, y_true):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = y_true, y_pred
    
    return torch.mean(torch.mean((_t - _p) ** 2 / (_t + 1e-06), axis = 1))

def total_loss(y_pred, y_true):
    xy_t, xy_p = y_true[:,:2], y_pred[:,:2]
    mv_t, mv_p = y_true[:,2:], y_pred[:,2:]
    
    e1 = torch.mean(torch.mean((xy_t - xy_p) ** 2, axis = 1)) / 2e+04
    e2 = torch.mean(torch.mean((mv_t - mv_p) ** 2 / (mv_t + 1e-06), axis = 1))
    
    return e1 + e2

def weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight)

In [4]:
def train_model(model, train_data, weight, optimizer, loss_func):
    loss_sum = 0
    for i, (x, y) in enumerate(train_data):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        pred = model(x)
        loss = loss_func(pred, y)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    
    return loss_sum / len(train_data)

In [5]:
def eval_model(model, val_data, loss_func):
    with torch.no_grad():
        loss = 0
        for i, (x, y) in enumerate(val_data):
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            loss += loss_func(pred, y).item()
    return loss / len(val_data)

## models

In [6]:
class conv_bn(nn.Module):
    def __init__(self, i_f, o_f, fs):
        super(conv_bn, self).__init__()
        self.conv = nn.Conv2d(i_f, o_f, fs)
        self.act = nn.ELU()
        self.bn = nn.BatchNorm2d(o_f)
        self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride= (2, 1))
    def forward(self, x):
        x = self.bn(self.act(self.conv(x)))
        return self.pool(x)

In [7]:
class conv_block(nn.Module):
    def __init__(self, h_list, input_shape, fs):
        '''
        input_shape : not include batch_size
        '''
        
        super(conv_block, self).__init__()
        self.input_shape = input_shape
        self.fs = fs
        convs = []
        for i in range(len(h_list)):
            if i == 0:
                convs.append(conv_bn(self.input_shape[0], h_list[i], fs))
            else:
                convs.append(conv_bn(h_list[i-1], h_list[i], fs))
        self.convs = nn.Sequential(*convs)
    
    def forward(self, x):
        return self.convs(x)

In [8]:
class classifier(nn.Module):
    def __init__(self, h_list, input_size, output_size):
        super(classifier, self).__init__()
        layers = []
        self.activation = nn.ELU()
        for i in range(len(h_list)):
            if i == 0:
                layers.append(nn.Linear(input_size, h_list[0]))
            else:
                layers.append(nn.Linear(h_list[i-1], h_list[i]))
            layers.append(nn.ELU())
            
        layers.append(nn.Linear(h_list[i], output_size))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

In [9]:
class cnn_model(nn.Module):
    def __init__(self, cnn_block, fc_block):
        super(cnn_model, self).__init__()
        self.cnn = cnn_block
        self.fc = fc_block
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(start_dim = 1)
        return self.fc(x)

- xy, m, v 쪼개서 학습 가능하게
- cross validation
- parallel cnn 구현
- lstm

### Configuration

In [10]:
EPOCH = 100
base_lr = 0.001
name = 'XYMV'
root_path = './model'
initialize = False

### load dataset

In [11]:
root_dir = '/home/bskim/project/kaeri/KAERI_dataset/'

train_f = pd.read_csv(os.path.join(root_dir, 'train_features.csv'))
train_t = pd.read_csv(os.path.join(root_dir, 'train_target.csv'))
test_f = pd.read_csv(os.path.join(root_dir, 'test_features.csv'))

train_f = train_f[['Time','S1','S2','S3','S4']].values
test_f = test_f[['Time','S1','S2','S3','S4']].values
train_f = train_f.reshape((-1, 1, 375, 5))#.astype(np.float32)
test_f = test_f.reshape((-1, 1, 375, 5))#.astype(np.float32)

train_target = train_t[list(name)].values#.astype(np.float32)
print(train_target.shape)
test_f = torch.FloatTensor(test_f)

(2800, 4)


### define model

In [12]:
fc = classifier([64], input_size = 512*3*5, output_size = 4)
conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, 4], (3, 1))

model = cnn_model(conv, fc)

optimizer = torch.optim.Adam(model.parameters(), lr = base_lr)
criterion = E1_loss
model = model.cuda()
if initialize:
    model.apply(weights_init)

In [13]:
summary(model, (1, 375, 5))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 373, 5]              64
               ELU-2           [-1, 16, 373, 5]               0
       BatchNorm2d-3           [-1, 16, 373, 5]              32
         MaxPool2d-4           [-1, 16, 186, 5]               0
           conv_bn-5           [-1, 16, 186, 5]               0
            Conv2d-6           [-1, 32, 184, 5]           1,568
               ELU-7           [-1, 32, 184, 5]               0
       BatchNorm2d-8           [-1, 32, 184, 5]              64
         MaxPool2d-9            [-1, 32, 92, 5]               0
          conv_bn-10            [-1, 32, 92, 5]               0
           Conv2d-11            [-1, 64, 90, 5]           6,208
              ELU-12            [-1, 64, 90, 5]               0
      BatchNorm2d-13            [-1, 64, 90, 5]             128
        MaxPool2d-14            [-1, 64

### train

In [14]:
trainx, valx, trainy, valy = train_test_split(train_f, train_target, test_size = 0.2, shuffle = True, random_state = 38)

train_dataset = dfDataset(trainx.astype(np.float32), trainy)
train_loader = DataLoader(train_dataset, batch_size = 256, shuffle = True)
val_dataset = dfDataset(valx.astype(np.float32), valy)
val_loader = DataLoader(val_dataset, batch_size = 256, shuffle = True)

In [15]:
val_losses = []
curr_loss = 1e+7
os.makedirs(root_path) if not os.path.exists(root_path) else None
for ep in range(1, EPOCH + 1):
    model.train()
    loss = train_model(model, train_loader, criterion, optimizer, criterion)
    model.eval()
    val_loss =eval_model(model, val_loader, criterion)
    if curr_loss > val_loss:
        print('[{}] : train loss {:4f}, val loss drop {:.4f} to {:.4f}'.format(ep, np.mean(loss), curr_loss, val_loss))
        curr_loss = val_loss
        torch.save(model.state_dict(), os.path.join(root_path, 'model_{}.pt'.format(name)))
    else:
        print('[{}] : train loss {:.4f}, val loss {:.4f}, not drop'.format(ep, np.mean(loss), val_loss))

[1] : train loss 1.616494, val loss drop 10000000.0000 to 1.6072
[2] : train loss 0.996160, val loss drop 1.6072 to 1.0413
[3] : train loss 0.433721, val loss drop 1.0413 to 0.3318
[4] : train loss 0.141878, val loss drop 0.3318 to 0.2052
[5] : train loss 0.056921, val loss drop 0.2052 to 0.0462
[6] : train loss 0.027530, val loss drop 0.0462 to 0.0334
[7] : train loss 0.016786, val loss drop 0.0334 to 0.0141
[8] : train loss 0.0109, val loss 0.0426, not drop
[9] : train loss 0.0077, val loss 0.0295, not drop
[10] : train loss 0.0057, val loss 0.0161, not drop
[11] : train loss 0.004488, val loss drop 0.0141 to 0.0073
[12] : train loss 0.0044, val loss 0.0101, not drop
[13] : train loss 0.0037, val loss 0.0171, not drop
[14] : train loss 0.002893, val loss drop 0.0073 to 0.0058
[15] : train loss 0.0029, val loss 0.0058, not drop
[16] : train loss 0.002622, val loss drop 0.0058 to 0.0036
[17] : train loss 0.002490, val loss drop 0.0036 to 0.0034
[18] : train loss 0.0021, val loss 0.0072

## test

In [25]:
model = cnn((3,1), len(name))
model.load_state_dict(torch.load('./tmp/model_XYMV.pt'))
model = model.cuda()

In [26]:
with torch.no_grad():
    predict = model(test_f.cuda())

In [27]:
submission = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
submission[['X','Y','M','V']] = predict.detach().cpu().numpy()
submission.to_csv(os.path.join(root_dir, 'cnn_20200625_total_loss.csv'), index = False)